In [ ]:
!git clone https://github.com/mhagiwara/universalscripts
%cd universalscripts

In [ ]:
%pip install Pillow
%pip install torch
%pip install torchvision

In [ ]:
%mkdir data/chars
%mkdir -p result/vae

In [ ]:
import torch
import torch.utils.data
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
from torchvision.utils import save_image
from PIL import Image
import PIL.ImageOps

In [ ]:
def split_chars():
    image = Image.open('data/chars.png')
    r, g, b, a = image.split()
    image = Image.merge('RGB', (r, g, b))

    image = PIL.ImageOps.invert(image)

    x_offset = 8
    y_offset = 6
    for x in range(40):
        for y in range(21):
            box = (x_offset + x * 48, y_offset + y * 48,
                   x_offset + x * 48 + 32, y_offset + y * 48 + 32)
            region = image.crop(box)
            region.save('data/chars/{:02d}{:02d}.png'.format(x, y))


In [ ]:
split_chars()

In [ ]:
IMG_WIDTH = 32
IMG_HEIGHT = 32
NUM_HIDDEN = 32

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)


class UnFlatten(nn.Module):
    def forward(self, input, size=1024):
        return input.view(input.size(0), size // 4, 2, 2)


class VAE(nn.Module):
    def __init__(self, image_channels=3, h_dim=1024, z_dim=32):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            Flatten()
        )

        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)

        self.decoder = nn.Sequential(
            UnFlatten(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, image_channels, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid(),
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + std * eps

    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def forward(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        z = self.fc3(z)
        return self.decoder(z), mu, logvar

In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    cross_entropy = F.binary_cross_entropy(
        recon_x, x.view(-1, IMG_WIDTH * IMG_HEIGHT), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    kl_distance = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return cross_entropy + kl_distance

In [ ]:
def generate_random(model, epoch):
    from torch.distributions.normal import Normal
    images = []
    for x in range(10+1):
        for y in range(10+1):

            z = torch.randn(NUM_HIDDEN).unsqueeze(0)
            recon = model.decoder(model.fc3(z))
            images.append(recon)

    images_joined = torch.cat(images).view(-1, 1, IMG_WIDTH, IMG_HEIGHT)
    save_image(images_joined.cpu(),
               'result/vae/epoch{:03d}.png'.format(epoch), nrow=11)

In [ ]:
def train_vae():
    seed = 1
    batch_size = 32
    epochs = 100
    log_interval = 10

    torch.manual_seed(seed)

    transform = transforms.Compose([transforms.Grayscale(),
                                    transforms.ToTensor()])
    chars = datasets.folder.ImageFolder('data/', transform=transform)
    train_loader = torch.utils.data.DataLoader(chars, batch_size=batch_size, shuffle=True)

    model = VAE(image_channels=1, h_dim=IMG_HEIGHT * IMG_WIDTH, z_dim=NUM_HIDDEN)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0
        for batch_idx, (data, _) in enumerate(train_loader):
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)
            loss = loss_function(recon_batch, data, mu, logvar)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader),
                    loss.item() / len(data)))

        print('====> Epoch: {} Average loss: {:.4f}'.format(
              epoch, train_loss / len(train_loader.dataset)))

        model.eval()
        generate_random(model, epoch)

    torch.save(model.state_dict(), 'result/vae/model.pt')


In [ ]:
train_vae()

In [ ]:
Image.open('result/vae/epoch080.png')